## Import modules

In [ ]:
!pip install imutils
!pip install loguru

In [ ]:
!cp ../input/vgg-simple/vgg.py /kaggle/working

In [ ]:
!pwd

In [ ]:
from vgg import *

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adam, Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, TerminateOnNaN, TensorBoard, ReduceLROnPlateau
from imutils import paths
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import argparse
import random
import pickle
import cv2
import os
import copy
import sys
from loguru import logger

In [ ]:
import logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

logger.debug("All modules imported")

In [ ]:
cur_dir = os.getcwd()
data_dir = os.path.join(cur_dir, "../input/apparel-images-dataset")
output_dir = os.path.join(cur_dir, "./")

logger.debug("[INFO] loading images on progres...")
data = []
labels = []

# Grab the image paths and shuffle them
imagePaths = sorted(list(paths.list_images(data_dir)))
random.seed(2)
random.shuffle(imagePaths)

IMAGE_WIDTH, IMAGE_HEIGHT = 64, 64

for imagePath in imagePaths:
    # Load the image
    image = cv2.imread(imagePath)
    
    # Resize it
    image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))
    
    # Append data
    data.append(image)
    
    # Extract the class label
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    
logger.debug("[INFO] data load complete...")

In [ ]:
row, col = 2,6
fig, axs = plt.subplots(row, col, figsize=(15,7))

count = 0
for r in range(row):
    for ax in axs[r]:
        ax.imshow(cv2.cvtColor(data[count], cv2.COLOR_BGR2RGB))
        ax.set_title(labels[count])
        ax.grid(False)
        count = count + 1
plt.show()

In [ ]:
# Scale the raw pixel intensities to the range [0,1]
data = np.array(data, dtype='float') / 255.0
labels = np.array(labels)

# Binarize labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# Save the encoder to output directory
with open(os.path.join(output_dir, "labels"), "wb") as f:
    pickle.dump(lb, f)
    
# Randomly split (15% test and 85% train)
train_X, test_X, train_y, test_y = train_test_split(data, labels, test_size=0.15, random_state=42)

In [ ]:
aug = ImageDataGenerator(rotation_range=45, width_shift_range=0.1,
                        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                        horizontal_flip=True, fill_mode="nearest")

# Initialize VGG
model = VGGNet.build(width=IMAGE_WIDTH, height=IMAGE_HEIGHT,
                    depth=3, classes=len(lb.classes_))

In [ ]:
# Initialize learning rate
INIT_LR = 0.0007
EPOCHS = 100
BS = 64

# Checkpoints between the training steps
model_checkpoint = ModelCheckpoint(filepath="VGG_epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5",
                                  monitor='val_loss',
                                  verbose=1,
                                  save_best_only=True,
                                  save_weights_only=False,
                                  mode='auto',
                                  period=20)

# Terminating of training if the loss become NaN
terminate_on_nan = TerminateOnNaN()

# Using tensorboard for visualization
t_board = TensorBoard(log_dir='./logs',
                      histogram_freq=0,
                      batch_size=32,
                      write_graph=True,
                      write_grads=False,
                      write_images=False,
                      embeddings_freq=0,
                      update_freq='epoch')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=10, min_lr=0.00001)

callbacks = [model_checkpoint, t_board, terminate_on_nan, reduce_lr]

# Initialize the model and optimizers
opt = Adam(lr=INIT_LR, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(loss='categorical_crossentropy', optimizer=opt,
              metrics=['accuracy'])

# Training the network
logger.debug("Training the network...")
H = model.fit_generator(aug.flow(train_X, train_y, batch_size=BS),
                        validation_data=(test_X, test_y), steps_per_epoch=len(train_X) // BS,
                        epochs=EPOCHS, callbacks=callbacks)

# Save the model locally for use later
model_path = os.path.join(output_dir, "trained_VGG_model.h5")
model.save(model_path)

In [ ]:
# Evaluate the network
logger.debug("Making predictions and evaluating the trained model")
predictions = model.predict(test_X, batch_size=32)
print(classification_report(test_y.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

# Plot the training loss and accuracy
n = np.arange(0, EPOCHS)
plt.figure()
plt.plot(N, H.history['loss'], label='train_loss')
plt.plot(N, H.history['val_loss'], label='val_loss')
plt.plot(N, H.history['acc'], label='train_acc')
plt.plot(N, H.history['val_acc'], label='val_acc')
plt.title("Training/Validation Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(os.path.join(output_dir, "vggnet_plot.png"))